In [1]:
import pandas as pd
import json
import os
print(os.listdir())

['titanic_notebook.ipynb', 'test.csv', 'train.csv', '.ipynb_checkpoints', 'gender_submission.csv']


In [2]:
#actual test data
test_df = pd.read_csv('test.csv')

#training set
train_df = pd.read_csv('train.csv')

#sample submission
gender_df = pd.read_csv('gender_submission.csv')

In [3]:
import numpy as np
import csv

with open('train.csv', 'rt') as f:
    data = list(csv.DictReader(f))
data[:1]

[OrderedDict([('PassengerId', '1'),
              ('Survived', '0'),
              ('Pclass', '3'),
              ('Name', 'Braund, Mr. Owen Harris'),
              ('Sex', 'male'),
              ('Age', '22'),
              ('SibSp', '1'),
              ('Parch', '0'),
              ('Ticket', 'A/5 21171'),
              ('Fare', '7.25'),
              ('Cabin', ''),
              ('Embarked', 'S')])]

In [4]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

_all_xs = [{k: v for k, v in row.items() if k != 'Survived'} for row in data]
_all_ys = np.array([int(row['Survived']) for row in data])

all_xs, all_ys = shuffle(_all_xs, _all_ys, random_state=0)
train_xs, valid_xs, train_ys, valid_ys = train_test_split(
    all_xs, all_ys, test_size=0.25, random_state=0)
print('{} items total, {:.1%} true'.format(len(all_xs), np.mean(all_ys)))

891 items total, 38.4% true


In [5]:
#minimal preprocessing: convert obviously contiuous Age and Fare variables to floats, 
# and SibSp, Parch to integers. Missing Age values are removed.

for x in all_xs:
    if x['Age']:
        x['Age'] = float(x['Age'])
    else:
        x.pop('Age')
    x['Fare'] = float(x['Fare'])
    x['SibSp'] = int(x['SibSp'])
    x['Parch'] = int(x['Parch'])

In [8]:
print(all_xs[:12])

[{'PassengerId': '496', 'Pclass': '3', 'Name': 'Yousseff, Mr. Gerious', 'Sex': 'male', 'SibSp': 0, 'Parch': 0, 'Ticket': '2627', 'Fare': 14.4583, 'Cabin': '', 'Embarked': 'C'}, {'PassengerId': '649', 'Pclass': '3', 'Name': 'Willey, Mr. Edward', 'Sex': 'male', 'SibSp': 0, 'Parch': 0, 'Ticket': 'S.O./P.P. 751', 'Fare': 7.55, 'Cabin': '', 'Embarked': 'S'}, {'PassengerId': '279', 'Pclass': '3', 'Name': 'Rice, Master. Eric', 'Sex': 'male', 'Age': 7.0, 'SibSp': 4, 'Parch': 1, 'Ticket': '382652', 'Fare': 29.125, 'Cabin': '', 'Embarked': 'Q'}, {'PassengerId': '32', 'Pclass': '1', 'Name': 'Spencer, Mrs. William Augustus (Marie Eugenie)', 'Sex': 'female', 'SibSp': 1, 'Parch': 0, 'Ticket': 'PC 17569', 'Fare': 146.5208, 'Cabin': 'B78', 'Embarked': 'C'}, {'PassengerId': '256', 'Pclass': '3', 'Name': 'Touma, Mrs. Darwis (Hanne Youssef Razi)', 'Sex': 'female', 'Age': 29.0, 'SibSp': 0, 'Parch': 2, 'Ticket': '2650', 'Fare': 15.2458, 'Cabin': '', 'Embarked': 'C'}, {'PassengerId': '299', 'Pclass': '1', '

## xgboost classifier   
very simple classifier with xbgoost.XGBClassifier and sklearn.feature_extraction.DictVectorizer,   
and check its accuracy with 10-fold cross-validation:

In [23]:
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

clf = XGBClassifier(use_label_encoder=False)
vec = DictVectorizer()
pipeline = make_pipeline(vec, clf)

def evaluate(_clf):
    scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(train_xs, train_ys, eval_metric='logloss', verbose=True)  # so that parts of the original pipeline are fitted

evaluate(pipeline)

[20:22:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607

ValueError: Pipeline.fit does not accept the eval_metric parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.

In [ ]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix('demo/data/agaricus.txt.train')
dtest = xgb.DMatrix('demo/data/agaricus.txt.test')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)